In [2]:
import xgboost
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import pymysql
import numpy as np
import math

from pickle import dump


import datetime
# import timedelta
# 오늘날짜 기준
dt = datetime.datetime.now()
fl_dt = float(str(dt.year) + "." + str(dt.month))

In [51]:
xgb_model = xgboost.XGBRegressor(n_estimators=5000, 
                                 learning_rate=0.2,
                                 rate_drop=0.1,
                                 max_depth =  10,
                                sample_type = 'uniform',
                                booster = 'dart')
# xgb_model.fit(res_dt.drop(columns=["rent_per_size"]), res_dt["rent_per_size"])
xgb_model.fit(train_data_scaled, price_data_scaled)

XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=8, num_parallel_tree=1, random_state=0,
             rate_drop=0.1, reg_alpha=0, reg_lambda=1, sample_type='uniform',
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [52]:
xgb_model

XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=8, num_parallel_tree=1, random_state=0,
             rate_drop=0.1, reg_alpha=0, reg_lambda=1, sample_type='uniform',
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [53]:
# res_dt_scale = res_dt_scale.drop(columns=[ 'predict_RDF_price_per_size', '100-MAPE_RDF'])

In [54]:
imp = (xgb_model.feature_importances_)

res_ft = pd.DataFrame({'col_nm' : train_data_scaled.columns.values, 'feature_importance' : imp})
res_ft = res_ft.sort_values(by=["feature_importance"], ascending = False)
res_ft

,col_nm,feature_importance
32,빌딩_5000,0.509963
45,강남역_소요시간,0.129430
44,개발지구_6000,0.043197
3,age,0.019122
8,rent_ratio,0.017499
28,close_apt_price_growth,0.017058
46,시청역_소요시간,0.016737
36,학교_500,0.015805
39,빌라_3000,0.015269
13,number_of_parking,0.011472


In [55]:
imp

array([0.00423198, 0.00512829, 0.00107903, 0.01912165, 0.        ,
       0.01094613, 0.00587014, 0.00320841, 0.01749947, 0.00292694,
       0.00169189, 0.00442342, 0.01079572, 0.0114717 , 0.00683237,
       0.00466918, 0.00303182, 0.00275879, 0.00246594, 0.00283699,
       0.00379079, 0.00513447, 0.00480753, 0.00385048, 0.00330153,
       0.00884693, 0.00530745, 0.0026837 , 0.01705826, 0.00571575,
       0.00698324, 0.00777446, 0.50996345, 0.00858385, 0.00985951,
       0.00536533, 0.01580492, 0.00532846, 0.00667542, 0.01526886,
       0.00483632, 0.00528797, 0.00668429, 0.00522697, 0.04319747,
       0.12942965, 0.01673662, 0.00490776, 0.00754439, 0.0030544 ],
      dtype=float32)

In [56]:
# res_dt = res_dt.drop(columns=["predict_RDF_price_per_size", "100-MAPE_RDF"])
train_data_scaled["predict_RDF_price_per_size"] = xgb_model.predict(train_data_scaled)
train_data_scaled["100-MAPE_RDF"] = 100 - ((abs(price_data_scaled["price_per_size_scaled"] - train_data_scaled["predict_RDF_price_per_size"]) / price_data_scaled["price_per_size_scaled"]) * 100)
train_data_scaled["100-MAPE_RDF"].mean()

-inf

In [57]:
train_data_scaled[train_data_scaled["100-MAPE_RDF"] == -np.inf]

,number_of_units_total,net_area,number_of_units,age,rebuild,number_of_dongs,floor_highest_dong,floor_lowest_dong,rent_ratio,kb_price_growth,...,오피스텔_6000,주상복합_5000,개발지구_6000,강남역_소요시간,시청역_소요시간,여의도역_소요시간,number_of_rooms,number_of_bathrooms,predict_RDF_price_per_size,100-MAPE_RDF
27963,0.015556,0.195249,0.009752,0.294118,0.0,0.008264,0.045455,0.086957,0.683606,0.123417,...,0.00994,0.012174,0.0,0.8,0.666667,0.675889,0.333333,0.0,0.013705,-inf


In [58]:
train_data_scaled[train_data_scaled["100-MAPE_RDF"] > 0]["100-MAPE_RDF"].mean()

94.8155093703067

In [338]:
orig_data_res = min_max_scaler.inverse_transform(res_dt_scale[res_dt_scale["100-MAPE_RDF"] != -np.inf].drop(columns=["predict_RDF_price_per_size","100-MAPE_RDF"]))
orig_data_res = pd.DataFrame(orig_data_res, columns = res_dt_scale[res_dt_scale["100-MAPE_RDF"] != -np.inf].drop(columns=["predict_RDF_price_per_size","100-MAPE_RDF"]).columns)
orig_data_res

,kb_unit_type_id,kb_apartment_id,number_of_units_total,net_area,number_of_units,age,number_of_dongs,floor_highest_dong,floor_lowest_dong,rent_ratio,...,IC_4000,IC_8000,오피스텔_6000,주상복합_5000,개발지구_6000,강남역_소요시간,시청역_소요시간,여의도역_소요시간,number_of_rooms,number_of_bathrooms
0,1.0,1.0,126.0,83.61,42.0,33.0,9.0,3.0,3.0,0.5529,...,11.0,57.0,5840.0,3471.0,20.0,58.0,23.0,39.0,3.0,1.0
1,1.0,1.0,126.0,83.61,42.0,33.0,9.0,3.0,3.0,0.5529,...,11.0,57.0,5840.0,3471.0,20.0,58.0,23.0,39.0,3.0,1.0
2,2.0,1.0,126.0,132.39,20.0,33.0,9.0,3.0,3.0,0.4957,...,11.0,57.0,5840.0,3471.0,20.0,58.0,23.0,39.0,4.0,2.0
3,2.0,1.0,126.0,132.39,20.0,33.0,9.0,3.0,3.0,0.4957,...,11.0,57.0,5840.0,3471.0,20.0,58.0,23.0,39.0,4.0,2.0
4,3.0,1.0,126.0,145.73,10.0,33.0,9.0,3.0,3.0,0.5000,...,11.0,57.0,5840.0,3471.0,20.0,58.0,23.0,39.0,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56259,50460.0,15276.0,120.0,84.73,60.0,17.0,1.0,15.0,15.0,0.7436,...,3.0,4.0,43.0,43.0,1.0,123.0,119.0,132.0,3.0,2.0
56260,50461.0,15277.0,280.0,62.74,140.0,5.0,4.0,19.0,17.0,0.7965,...,3.0,4.0,43.0,43.0,4.0,124.0,120.0,133.0,3.0,2.0
56261,50461.0,15277.0,280.0,62.74,140.0,5.0,4.0,19.0,17.0,0.7965,...,3.0,4.0,43.0,43.0,4.0,124.0,120.0,133.0,3.0,2.0
56262,50462.0,15277.0,280.0,76.11,140.0,5.0,4.0,19.0,17.0,0.7538,...,3.0,4.0,43.0,43.0,4.0,124.0,120.0,133.0,3.0,2.0


In [59]:
import pickle

import datetime
# import timedelta
# 오늘날짜 기준
dt = datetime.datetime.now()
fl_dt = str(dt.year) + str(dt.month) + str(dt.day)

pickle.dump(xgb_model, open('xgb_model_수도권v0.86(매매가)' + fl_dt, 'wb'))

In [60]:
load_md = pickle.load(open('xgb_model_수도권v0.86(매매가)' + fl_dt,'rb'))
load_md

XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=8, num_parallel_tree=1, random_state=0,
             rate_drop=0.1, reg_alpha=0, reg_lambda=1, sample_type='uniform',
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [61]:
ft_nm = np.array(load_md.get_booster().feature_names)
ft_nm

array(['number_of_units_total', 'net_area', 'number_of_units', 'age',
       'rebuild', 'number_of_dongs', 'floor_highest_dong',
       'floor_lowest_dong', 'rent_ratio', 'kb_price_growth',
       'kb_rent_growth', 'x', 'y', 'number_of_parking', 'academy',
       'harmful_facility', 'subway_index', 'min_subway_distance',
       'max_subway_distance', 'mean_subway_distance', 'subway', 'culture',
       'department_store', 'mart', 'cafe', 'convenience', 'hospital',
       'pharmacy', 'close_apt_price_growth', 'close_apt_rent_growth',
       'kb_rent_growth_24', 'kb_rent_growth_36', '빌딩_5000', '상가_100',
       '상가_200', '학교_3000', '학교_500', '도서관_3000', '우체국_5000', '빌라_3000',
       'IC_4000', 'IC_8000', '오피스텔_6000', '주상복합_5000', '개발지구_6000',
       '강남역_소요시간', '시청역_소요시간', '여의도역_소요시간', 'number_of_rooms',
       'number_of_bathrooms'], dtype='<U22')

In [332]:
# X_train.iloc[0].values

In [333]:
# vl_dt = np.array([3.56670000e+04, 1.05970000e+04, 8.30000000e+01, 2.08000000e+02,
#        4.00000000e+00, 5.39500000e-01, 1.59400000e+01, 3.19300000e+01,
#        1.27141200e+02, 3.74679197e+01, 3.01000000e+02, 3.36000000e+02,
#        1.00000000e+00, 0.00000000e+00, 2.60000000e+01, 5.00000000e+00,
#        6.80000000e+01, 7.40000000e+01, 1.70000000e+01, 7.50000000e+01,
#        1.10000000e+01, 0.00000000e+00, 2.46373653e+01, 3.09224476e+01,
#        5.54500000e+01, 5.54500000e+01, 2.08800000e+03, 3.00000000e+00,
#        2.00000000e+00])

# rslt_dt = pd.DataFrame(vl_dt.reshape(1,-1),columns=ft_nm)
# rslt_dt

In [334]:
# load_md.predict(rslt_dt)[0]

In [62]:
from datetime import date
import os
#     import os

tdy = str(date.today().isoformat()).replace('-','')
fl_nm = 'train_raw_data'

dir = 'D:\\ML model\\' + 'ML_'+ tdy
if not os.path.exists(dir):
    os.mkdir(dir)

dir_nm = os.path.join(dir, fl_nm)

res_dt.to_csv(dir_nm +".csv", index = False)

In [63]:
tdy

'20211201'

In [105]:
final_data = pd.read_csv('D:\\ML model\\ML_20211201\\train_raw_data.csv')
final_data

,kb_unit_type_id,kb_apartment_id,number_of_units_total,net_area,number_of_units,age,rebuild,number_of_dongs,floor_highest_dong,floor_lowest_dong,...,IC_8000,오피스텔_6000,주상복합_5000,개발지구_6000,강남역_소요시간,시청역_소요시간,여의도역_소요시간,number_of_rooms,number_of_bathrooms,price_per_size
0,1,1,126,83.61,42,33.0,1,9,3,3,...,57,5840,3471,20,58,23,39,3,1,3360.743033
1,2,1,126,132.39,20,33.0,1,9,3,3,...,57,5840,3471,20,58,23,39,4,2,2871.555820
2,3,1,126,145.73,10,33.0,1,9,3,3,...,57,5840,3471,20,58,23,39,5,2,2722.117615
3,4,2,60,84.51,20,21.0,0,4,5,5,...,65,6216,3574,18,56,21,35,3,2,3442.303633
4,5,2,60,129.76,20,21.0,0,4,5,5,...,65,6216,3574,18,56,21,35,4,2,3184.518534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28189,50458,15276,120,59.74,30,17.0,0,1,15,15,...,4,43,43,1,123,119,132,3,1,816.209052
28190,50459,15276,120,59.74,30,17.0,0,1,15,15,...,4,43,43,1,123,119,132,3,1,816.209052
28191,50460,15276,120,84.73,60,17.0,0,1,15,15,...,4,43,43,1,123,119,132,3,2,760.802638
28192,50461,15277,280,62.74,140,5.0,0,4,19,17,...,4,43,43,4,124,120,133,3,2,1488.498984


In [114]:
# test_dt = pd.read_csv(dir + '\\' + 'train_raw_data.csv')
test_dt = final_data

In [115]:
min_max_scaler1  = MinMaxScaler()
min_max_scaler2  = MinMaxScaler()

min_max_scaler1.fit_transform(test_dt.drop(columns=["kb_unit_type_id", "kb_apartment_id","price_per_size"]))

test_dt_scale = min_max_scaler1.fit_transform(test_dt.drop(columns=["price_per_size"]))
test_dt_scale = pd.DataFrame(test_dt_scale, columns = test_dt.drop(columns=["price_per_size"]).columns)

min_max_scaler2.fit_transform(test_dt[["kb_unit_type_id","price_per_size"]])

test_dt_scale["predict_xgb_price_per_size"] = load_md.predict(test_dt_scale.drop(columns=["kb_unit_type_id", "kb_apartment_id"]))

In [116]:
min_max_scaler1 = min_max_scaler1.fit(test_dt.drop(columns=["kb_unit_type_id", "kb_apartment_id","price_per_size"]))
min_max_scaler1

MinMaxScaler()

In [117]:
from pickle import dump

dump(min_max_scaler1, open('input_scaler_re.pkl', 'wb'))

In [118]:

min_max_scaler2  = MinMaxScaler()

min_max_scaler2.fit_transform(test_dt[["price_per_size"]])

# test_dt_scale["predict_xgb_price_per_size"] = load_md.predict(test_dt_scale.drop(columns=["kb_unit_type_id", "kb_apartment_id"]))

array([[0.20058908],
       [0.16892312],
       [0.15924972],
       ...,
       [0.03229032],
       [0.0793954 ],
       [0.07441856]])

In [119]:
min_max_scaler2 = min_max_scaler2.fit(test_dt[["price_per_size"]])
min_max_scaler2

MinMaxScaler()

In [120]:
# import joblib
from pickle import dump

# scaler_filename1 = "input_scaler.save"
# joblib.dump(min_max_scaler1, scaler_filename1) 


dump(min_max_scaler1, open('input_scaler.pkl', 'wb'))



# scaler_filename2 = "output_scaler.save"
# joblib.dump(min_max_scaler2, scaler_filename2) 

dump(min_max_scaler2, open('output_scaler.pkl', 'wb'))


In [121]:
final_result = min_max_scaler2.inverse_transform(test_dt_scale[["predict_xgb_price_per_size"]])
final_result = pd.DataFrame(final_result, columns = ["predict_xgb_price_per_size"])
# final_result['kb_unit_type_id'] = final_result['kb_unit_type_id'].astype(int)
final_result

,predict_xgb_price_per_size
0,3255.034180
1,2826.973877
2,2755.410156
3,3298.311523
4,3114.903564
...,...
28189,812.081909
28190,808.190125
28191,802.536011
28192,1393.265137


In [124]:
final_data["100-MAPE_RDF"] = 100 - ((abs(final_data["price_per_size"] - final_data["predict_xgb_price_per_size"]) / final_data["price_per_size"]) * 100)
final_data["100-MAPE_RDF"].mean()

95.77642420683833

In [130]:
final_data_addr2['100-MAPE_RDF'].mean()

95.76450422147848

In [132]:
final_data_addr2.to_csv('D:\\ML model\\ML_20211201\\final_addr.csv', index=False, encoding = 'CP949')

In [67]:
final_data_addr.groupby(final_data_addr['province_name'])['100-MAPE_RDF'].mean()

province_name
경기도      89.485559
서울특별시    92.407984
인천광역시    88.256039
Name: 100-MAPE_RDF, dtype: float64

In [189]:
final_data_addr[final_data_addr["city_name"] == "강남구"].sort_values( by = 'number_of_units', ascending=False).head(10)["100-MAPE_RDF"].mean()

96.83022684709984

In [53]:
final_data_addr[final_data_addr["city_name"] == "강남구"].sort_values( by = 'number_of_units', ascending=False).head(10)["100-MAPE_RDF"].mean()

95.90181277229036

In [54]:
final_data_addr[final_data_addr["city_name"] == "강남구"].sort_values( by = '100-MAPE_RDF', ascending=False).head(10)["100-MAPE_RDF"].mean()

99.94162667293224